In [8]:
gt_series = 'twitter_time_series_to_8_03.json'
end_date = '8_31'
compared = [f'news_westclass_time_series_to_{end_date}.json',
           f'news_string_match_time_series_to_{end_date}.json',
           f'news_leidos_bert_time_series_to_{end_date}.json',
           f'news_lotclass_time_series_to_{end_date}.json',
           f'news_tuned_bert_time_series_to_{end_date}.json']
narratives = ['benefits/connections/afghanistan', 'benefits/covid', 'benefits/development/energy', 'benefits/development/maritime', 'benefits/development/roads', 'benefits/jobs', 'controversies/china/border', 'controversies/china/debt', 'controversies/china/exploitation', 'controversies/china/funding', 'controversies/china/naval', 'controversies/china/uighur', 'controversies/pakistan/army', 'controversies/pakistan/bajwa', 'controversies/pakistan/baloch', 'controversies/pakistan/students', 'leadership/bajwa', 'leadership/khan', 'leadership/sharif', 'opposition/kashmir', 'opposition/propaganda']
import calendar
import numpy as np
import datetime
import json
import pandas as pd
from scipy.stats import pearsonr

evaluate_dates = [i.strftime("%Y-%m-%d") for i in pd.date_range(start="2020-03-30",end="2020-06-29").to_pydatetime().tolist()]


In [9]:
import math

In [10]:
gt_all = pd.read_json('../' + gt_series, typ='series')
print(' ' + ' '.join(map(lambda x: '_'.join(x.split('_')[1:-5]), compared)))
nar2method_corr = dict()
for nar in narratives:
    nar2method_corr[nar] = dict()
    print(nar, end=' ')
    for method in compared:
        a = json.loads(pd.read_json('../generate_output/' + method, typ='series')[nar])
        gt = json.loads(gt_all[nar])['EventCount']
        X = []
        Y = []
        for d in evaluate_dates:
            stamp = str(int(calendar.timegm(datetime.datetime.strptime(d, "%Y-%m-%d").timetuple()))) + '000'
            X.append(a[stamp])
            Y.append(gt[stamp])
        corr, p = pearsonr(X, Y)
        nar2method_corr[nar][method] = corr if not math.isnan(corr) else -100
        print(corr, end=' ')
    print()

 westclass string_match leidos_bert lotclass tuned_bert
benefits/connections/afghanistan 0.36896064233135034 0.47903696253600037 0.2728649252791227 -0.07021458269544452 0.6450430104226977 
benefits/covid 0.14199851695920154 0.1252692423531799 0.04111000332146582 0.07336412556325703 0.1370396216279329 
benefits/development/energy -0.027119588202836206 -0.03186845902957903 0.24282662404719516 0.047901031029797064 0.15296712812934676 
benefits/development/maritime 0.03574264016959078 0.001238397142131071 0.32391500564113446 -0.024947843316250985 0.2491868818891263 
benefits/development/roads -0.11389325666188815 -0.09400323736444885 -0.0640255749328773 0.2503651345162133 0.08400182669052159 
benefits/jobs 0.0866010418770056 0.008185864933823762 0.11213910648550465 0.19298461819238819 -0.038447990535634836 
controversies/china/border 0.9257805417119256 0.9330215571967652 0.925680369405092 0.8892646951789432 0.7953705248769093 
controversies/china/debt 0.2615422275713903 0.2520863518951694 

In [11]:
nar2optimal_choice = dict()
for nar in narratives:
    res = nar2method_corr[nar]
    rk = sorted(res, key=res.get, reverse=True)
    x = rk[0]
    x = '_'.join(x.split('_')[1:-5])
    nar2optimal_choice[nar] = x
    print(x)
print(json.dumps(nar2optimal_choice))
# nar2optimal_choice = {"benefits/connections/afghanistan": "tuned_bert", "benefits/covid": "westclass", "benefits/development/energy": "leidos_bert", "benefits/development/maritime": "leidos_bert", "benefits/development/roads": "lotclass", "benefits/jobs": "lotclass", "controversies/china/border": "westclass", "controversies/china/debt": "string_match", "controversies/china/exploitation": "tuned_bert", "controversies/china/funding": "leidos_bert", "controversies/china/naval": "tuned_bert", "controversies/china/uighur": "string_match", "controversies/pakistan/army": "tuned_bert", "controversies/pakistan/bajwa": "tuned_bert", "controversies/pakistan/baloch": "westclass", "controversies/pakistan/students": "lotclass", "leadership/bajwa": "tuned_bert", "leadership/khan": "leidos_bert", "leadership/sharif": "tuned_bert", "opposition/kashmir": "leidos_bert", "opposition/propaganda": "leidos_bert"}
# nar2optimal_choice = {"benefits/connections/afghanistan": "tuned_bert", "benefits/covid": "westclass", "benefits/development/energy": "leidos_bert", "benefits/development/maritime": "leidos_bert", "benefits/development/roads": "lotclass", "benefits/jobs": "lotclass", "controversies/china/border": "string_match", "controversies/china/debt": "westclass", "controversies/china/exploitation": "tuned_bert", "controversies/china/funding": "leidos_bert", "controversies/china/naval": "tuned_bert", "controversies/china/uighur": "westclass", "controversies/pakistan/army": "tuned_bert", "controversies/pakistan/bajwa": "tuned_bert", "controversies/pakistan/baloch": "westclass", "controversies/pakistan/students": "lotclass", "leadership/bajwa": "tuned_bert", "leadership/khan": "leidos_bert", "leadership/sharif": "tuned_bert", "opposition/kashmir": "string_match", "opposition/propaganda": "string_match"}
# nar2optimal_choice = {"benefits/connections/afghanistan": "tuned_bert", "benefits/covid": "westclass", "benefits/development/energy": "leidos_bert", "benefits/development/maritime": "leidos_bert", "benefits/development/roads": "lotclass", "benefits/jobs": "lotclass", "controversies/china/border": "string_match", "controversies/china/debt": "westclass", "controversies/china/exploitation": "tuned_bert", "controversies/china/funding": "leidos_bert", "controversies/china/naval": "tuned_bert", "controversies/china/uighur": "westclass", "controversies/pakistan/army": "tuned_bert", "controversies/pakistan/bajwa": "tuned_bert", "controversies/pakistan/baloch": "westclass", "controversies/pakistan/students": "lotclass", "leadership/bajwa": "tuned_bert", "leadership/khan": "leidos_bert", "leadership/sharif": "tuned_bert", "opposition/kashmir": "string_match", "opposition/propaganda": "string_match"}
# nar2optimal_choice = {"benefits/connections/afghanistan": "tuned_bert", "benefits/covid": "westclass", "benefits/development/energy": "leidos_bert", "benefits/development/maritime": "leidos_bert", "benefits/development/roads": "lotclass", "benefits/jobs": "lotclass", "controversies/china/border": "string_match", "controversies/china/debt": "westclass", "controversies/china/exploitation": "tuned_bert", "controversies/china/funding": "leidos_bert", "controversies/china/naval": "tuned_bert", "controversies/china/uighur": "westclass", "controversies/pakistan/army": "tuned_bert", "controversies/pakistan/bajwa": "tuned_bert", "controversies/pakistan/baloch": "westclass", "controversies/pakistan/students": "lotclass", "leadership/bajwa": "tuned_bert", "leadership/khan": "leidos_bert", "leadership/sharif": "tuned_bert", "opposition/kashmir": "string_match", "opposition/propaganda": "string_match"}
nar2optimal_choice = {"benefits/connections/afghanistan": "tuned_bert", "benefits/covid": "westclass", "benefits/development/energy": "leidos_bert", "benefits/development/maritime": "leidos_bert", "benefits/development/roads": "lotclass", "benefits/jobs": "lotclass", "controversies/china/border": "string_match", "controversies/china/debt": "westclass", "controversies/china/exploitation": "tuned_bert", "controversies/china/funding": "leidos_bert", "controversies/china/naval": "tuned_bert", "controversies/china/uighur": "westclass", "controversies/pakistan/army": "tuned_bert", "controversies/pakistan/bajwa": "tuned_bert", "controversies/pakistan/baloch": "westclass", "controversies/pakistan/students": "lotclass", "leadership/bajwa": "tuned_bert", "leadership/khan": "leidos_bert", "leadership/sharif": "tuned_bert", "opposition/kashmir": "string_match", "opposition/propaganda": "string_match"}

tuned_bert
westclass
leidos_bert
leidos_bert
lotclass
lotclass
string_match
westclass
tuned_bert
leidos_bert
tuned_bert
westclass
tuned_bert
tuned_bert
westclass
lotclass
tuned_bert
leidos_bert
tuned_bert
string_match
string_match
{"benefits/connections/afghanistan": "tuned_bert", "benefits/covid": "westclass", "benefits/development/energy": "leidos_bert", "benefits/development/maritime": "leidos_bert", "benefits/development/roads": "lotclass", "benefits/jobs": "lotclass", "controversies/china/border": "string_match", "controversies/china/debt": "westclass", "controversies/china/exploitation": "tuned_bert", "controversies/china/funding": "leidos_bert", "controversies/china/naval": "tuned_bert", "controversies/china/uighur": "westclass", "controversies/pakistan/army": "tuned_bert", "controversies/pakistan/bajwa": "tuned_bert", "controversies/pakistan/baloch": "westclass", "controversies/pakistan/students": "lotclass", "leadership/bajwa": "tuned_bert", "leadership/khan": "leidos_bert", "

In [12]:
gt_all = pd.read_json('../' + gt_series, typ='series')
a = [pd.read_json('../generate_output/' + method, typ='series') for method in compared]
# print('_' + '_'.join(map(lambda x: ' '.join(x.split('_')[1:-5]), compared)))
nar2weights = dict()
import itertools
from multiprocessing import Pool
tmp = list(itertools.product(range(11), repeat=len(compared)))
search_space = []
for ratio in tmp:
    if sum(ratio) == 0:
        continue
    g = ratio[0]
    for r in ratio:
        g = math.gcd(g, r)
    if g != 1:
        continue
    search_space.append(ratio)
print(len(search_space))
def worker(data_in):
    nar, ratio = data_in
    gt = json.loads(gt_all[nar])['EventCount']
    X = []
    Y = []
    for d in evaluate_dates:
        stamp = str(int(calendar.timegm(datetime.datetime.strptime(d, "%Y-%m-%d").timetuple()))) + '000'
        val = 0
        base = 0
        assert len(ratio) == len(a)
        for r, value in zip(ratio, a):
            val += json.loads(value[nar])[stamp] * r
            base += r
        X.append(val / base)
        Y.append(gt[stamp])
    corr, p = pearsonr(X, Y)
    return corr, ratio
from tqdm.notebook import tqdm
for nar in narratives:
#     print(nar, end='_')
    corrs = []
    with Pool(128) as pool:
        for corr, ratio in pool.imap(worker, [(nar, ratio) for ratio in search_space]):
            corrs.append((corr if not math.isnan(corr) else -100, ' '.join(map(str, ratio))))
    corrs = sorted(corrs, reverse=True)
    print(nar, corrs[0][0], corrs[0][1])
    nar2weights[nar] = {j:i for i, j in zip(map(int, corrs[0][1].split(' ')), compared)}

152041
benefits/connections/afghanistan 0.648987444556655 0 0 1 0 10
benefits/covid 0.151608697789714 9 0 0 0 10
benefits/development/energy 0.24282662404719516 0 0 1 0 0
benefits/development/maritime 0.32391500564113446 0 0 1 0 0
benefits/development/roads 0.2503651345162133 0 0 0 1 0
benefits/jobs 0.19791902253994684 1 0 1 10 0
controversies/china/border 0.9442021676522102 0 1 1 5 0
controversies/china/debt 0.29482357507791856 3 0 6 8 0
controversies/china/exploitation 0.5240301892139482 0 0 0 9 10
controversies/china/funding 0.06940952705562402 0 0 7 0 6
controversies/china/naval 0.19155199779952292 0 0 8 2 5
controversies/china/uighur 0.4247198670795459 10 0 0 2 1
controversies/pakistan/army 0.36426082560192274 9 1 8 9 3
controversies/pakistan/bajwa 0.4139915678461167 0 0 0 9 7
controversies/pakistan/baloch 0.20280865091191722 3 0 1 4 0
controversies/pakistan/students 0.24131361673287832 0 1 0 10 0
leadership/bajwa 0.13728208888431623 1 3 0 0 6
leadership/khan 0.4951940422243769 10

In [13]:
json.dumps({nar: {'_'.join(method.split('_')[1:-5]) : nar2weights[nar][method] for method in nar2weights[nar]} for nar in nar2weights})
# {"benefits/connections/afghanistan": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 10}, "benefits/covid": {"westclass": 1, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "benefits/development/energy": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/maritime": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/roads": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}, "benefits/jobs": {"westclass": 1, "string_match": 0, "leidos_bert": 1, "lotclass": 10, "tuned_bert": 0}, "controversies/china/border": {"westclass": 0, "string_match": 1, "leidos_bert": 1, "lotclass": 5, "tuned_bert": 0}, "controversies/china/debt": {"westclass": 3, "string_match": 0, "leidos_bert": 6, "lotclass": 10, "tuned_bert": 0}, "controversies/china/exploitation": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 7, "tuned_bert": 10}, "controversies/china/funding": {"westclass": 0, "string_match": 0, "leidos_bert": 6, "lotclass": 0, "tuned_bert": 5}, "controversies/china/naval": {"westclass": 0, "string_match": 0, "leidos_bert": 8, "lotclass": 2, "tuned_bert": 5}, "controversies/china/uighur": {"westclass": 10, "string_match": 0, "leidos_bert": 0, "lotclass": 3, "tuned_bert": 1}, "controversies/pakistan/army": {"westclass": 9, "string_match": 1, "leidos_bert": 8, "lotclass": 9, "tuned_bert": 3}, "controversies/pakistan/bajwa": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 7}, "controversies/pakistan/baloch": {"westclass": 10, "string_match": 0, "leidos_bert": 3, "lotclass": 2, "tuned_bert": 0}, "controversies/pakistan/students": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 10, "tuned_bert": 0}, "leadership/bajwa": {"westclass": 3, "string_match": 8, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 9}, "leadership/khan": {"westclass": 9, "string_match": 0, "leidos_bert": 7, "lotclass": 9, "tuned_bert": 2}, "leadership/sharif": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "opposition/kashmir": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 0}, "opposition/propaganda": {"westclass": 10, "string_match": 10, "leidos_bert": 1, "lotclass": 1, "tuned_bert": 0}}
# {"benefits/connections/afghanistan": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 10}, "benefits/covid": {"westclass": 1, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "benefits/development/energy": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/maritime": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/roads": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}, "benefits/jobs": {"westclass": 1, "string_match": 0, "leidos_bert": 1, "lotclass": 10, "tuned_bert": 0}, "controversies/china/border": {"westclass": 0, "string_match": 1, "leidos_bert": 1, "lotclass": 5, "tuned_bert": 0}, "controversies/china/debt": {"westclass": 3, "string_match": 0, "leidos_bert": 6, "lotclass": 10, "tuned_bert": 0}, "controversies/china/exploitation": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 10}, "controversies/china/funding": {"westclass": 0, "string_match": 0, "leidos_bert": 7, "lotclass": 0, "tuned_bert": 6}, "controversies/china/naval": {"westclass": 0, "string_match": 0, "leidos_bert": 8, "lotclass": 2, "tuned_bert": 5}, "controversies/china/uighur": {"westclass": 10, "string_match": 0, "leidos_bert": 0, "lotclass": 2, "tuned_bert": 1}, "controversies/pakistan/army": {"westclass": 9, "string_match": 1, "leidos_bert": 8, "lotclass": 9, "tuned_bert": 3}, "controversies/pakistan/bajwa": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 7}, "controversies/pakistan/baloch": {"westclass": 10, "string_match": 0, "leidos_bert": 3, "lotclass": 2, "tuned_bert": 0}, "controversies/pakistan/students": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 10, "tuned_bert": 0}, "leadership/bajwa": {"westclass": 3, "string_match": 4, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 8}, "leadership/khan": {"westclass": 9, "string_match": 0, "leidos_bert": 7, "lotclass": 9, "tuned_bert": 2}, "leadership/sharif": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "opposition/kashmir": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 0}, "opposition/propaganda": {"westclass": 7, "string_match": 10, "leidos_bert": 1, "lotclass": 2, "tuned_bert": 0}}
# {"benefits/connections/afghanistan": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 10}, "benefits/covid": {"westclass": 1, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "benefits/development/energy": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/maritime": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/roads": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}, "benefits/jobs": {"westclass": 1, "string_match": 0, "leidos_bert": 1, "lotclass": 10, "tuned_bert": 0}, "controversies/china/border": {"westclass": 0, "string_match": 1, "leidos_bert": 1, "lotclass": 5, "tuned_bert": 0}, "controversies/china/debt": {"westclass": 3, "string_match": 0, "leidos_bert": 6, "lotclass": 10, "tuned_bert": 0}, "controversies/china/exploitation": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 7, "tuned_bert": 8}, "controversies/china/funding": {"westclass": 0, "string_match": 0, "leidos_bert": 7, "lotclass": 0, "tuned_bert": 6}, "controversies/china/naval": {"westclass": 0, "string_match": 0, "leidos_bert": 8, "lotclass": 2, "tuned_bert": 5}, "controversies/china/uighur": {"westclass": 10, "string_match": 0, "leidos_bert": 0, "lotclass": 2, "tuned_bert": 1}, "controversies/pakistan/army": {"westclass": 9, "string_match": 1, "leidos_bert": 8, "lotclass": 9, "tuned_bert": 3}, "controversies/pakistan/bajwa": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 7}, "controversies/pakistan/baloch": {"westclass": 10, "string_match": 0, "leidos_bert": 3, "lotclass": 8, "tuned_bert": 0}, "controversies/pakistan/students": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 10, "tuned_bert": 0}, "leadership/bajwa": {"westclass": 3, "string_match": 4, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 8}, "leadership/khan": {"westclass": 10, "string_match": 0, "leidos_bert": 1, "lotclass": 8, "tuned_bert": 0}, "leadership/sharif": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "opposition/kashmir": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 0}, "opposition/propaganda": {"westclass": 0, "string_match": 10, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}}
# {"benefits/connections/afghanistan": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 10}, "benefits/covid": {"westclass": 1, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "benefits/development/energy": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/maritime": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/roads": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}, "benefits/jobs": {"westclass": 1, "string_match": 0, "leidos_bert": 1, "lotclass": 10, "tuned_bert": 0}, "controversies/china/border": {"westclass": 0, "string_match": 1, "leidos_bert": 1, "lotclass": 5, "tuned_bert": 0}, "controversies/china/debt": {"westclass": 3, "string_match": 0, "leidos_bert": 6, "lotclass": 10, "tuned_bert": 0}, "controversies/china/exploitation": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 7, "tuned_bert": 8}, "controversies/china/funding": {"westclass": 0, "string_match": 0, "leidos_bert": 7, "lotclass": 0, "tuned_bert": 6}, "controversies/china/naval": {"westclass": 0, "string_match": 0, "leidos_bert": 8, "lotclass": 2, "tuned_bert": 5}, "controversies/china/uighur": {"westclass": 10, "string_match": 0, "leidos_bert": 0, "lotclass": 2, "tuned_bert": 1}, "controversies/pakistan/army": {"westclass": 9, "string_match": 1, "leidos_bert": 8, "lotclass": 10, "tuned_bert": 3}, "controversies/pakistan/bajwa": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 7}, "controversies/pakistan/baloch": {"westclass": 10, "string_match": 0, "leidos_bert": 3, "lotclass": 8, "tuned_bert": 0}, "controversies/pakistan/students": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 10, "tuned_bert": 0}, "leadership/bajwa": {"westclass": 3, "string_match": 3, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 10}, "leadership/khan": {"westclass": 10, "string_match": 0, "leidos_bert": 1, "lotclass": 8, "tuned_bert": 0}, "leadership/sharif": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "opposition/kashmir": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 0}, "opposition/propaganda": {"westclass": 0, "string_match": 10, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}}
# {"benefits/connections/afghanistan": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 10}, "benefits/covid": {"westclass": 9, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 10}, "benefits/development/energy": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/maritime": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/roads": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}, "benefits/jobs": {"westclass": 1, "string_match": 0, "leidos_bert": 1, "lotclass": 10, "tuned_bert": 0}, "controversies/china/border": {"westclass": 0, "string_match": 1, "leidos_bert": 1, "lotclass": 5, "tuned_bert": 0}, "controversies/china/debt": {"westclass": 3, "string_match": 0, "leidos_bert": 6, "lotclass": 8, "tuned_bert": 0}, "controversies/china/exploitation": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 10}, "controversies/china/funding": {"westclass": 0, "string_match": 0, "leidos_bert": 7, "lotclass": 0, "tuned_bert": 6}, "controversies/china/naval": {"westclass": 0, "string_match": 0, "leidos_bert": 8, "lotclass": 2, "tuned_bert": 5}, "controversies/china/uighur": {"westclass": 10, "string_match": 0, "leidos_bert": 0, "lotclass": 2, "tuned_bert": 1}, "controversies/pakistan/army": {"westclass": 9, "string_match": 1, "leidos_bert": 8, "lotclass": 9, "tuned_bert": 3}, "controversies/pakistan/bajwa": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 9, "tuned_bert": 7}, "controversies/pakistan/baloch": {"westclass": 3, "string_match": 0, "leidos_bert": 1, "lotclass": 4, "tuned_bert": 0}, "controversies/pakistan/students": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 10, "tuned_bert": 0}, "leadership/bajwa": {"westclass": 1, "string_match": 3, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 6}, "leadership/khan": {"westclass": 10, "string_match": 0, "leidos_bert": 1, "lotclass": 1, "tuned_bert": 0}, "leadership/sharif": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 1}, "opposition/kashmir": {"westclass": 0, "string_match": 1, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 0}, "opposition/propaganda": {"westclass": 0, "string_match": 10, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}}

'{"benefits/connections/afghanistan": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 10}, "benefits/covid": {"westclass": 9, "string_match": 0, "leidos_bert": 0, "lotclass": 0, "tuned_bert": 10}, "benefits/development/energy": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/maritime": {"westclass": 0, "string_match": 0, "leidos_bert": 1, "lotclass": 0, "tuned_bert": 0}, "benefits/development/roads": {"westclass": 0, "string_match": 0, "leidos_bert": 0, "lotclass": 1, "tuned_bert": 0}, "benefits/jobs": {"westclass": 1, "string_match": 0, "leidos_bert": 1, "lotclass": 10, "tuned_bert": 0}, "controversies/china/border": {"westclass": 0, "string_match": 1, "leidos_bert": 1, "lotclass": 5, "tuned_bert": 0}, "controversies/china/debt": {"westclass": 3, "string_match": 0, "leidos_bert": 6, "lotclass": 8, "tuned_bert": 0}, "controversies/china/exploitation": {"westclass": 0, "string_match": 0, "lei